To demonstrate the ``dPULearn().fit()``method, we create a small example dataset containing positive (1) and unlabeled (2) data samples:

In [1]:
import aaanalysis as aa
import pandas as pd
import numpy as np
aa.options["verbose"] = False

X = np.array([[0.2, 0.1], [0.25, 0.2], [0.2, 0.3], [0.5, 0.7]])
labels = np.array([1, 2, 2, 2])

Use ``dPULearn`` with default Principal Component Analysis (PCA) to obtain a defined number of reliable negatives samples (0) by specifying the ``n_unl_to_neg`` parameter:

In [2]:
dpul = aa.dPULearn()
dpul.fit(X=X, labels=labels, n_unl_to_neg=1)
df_pu = dpul.df_pu_
labels = dpul.labels_ # Updated labels
aa.display_df(df_pu)

,selection_via,PC1 (100.0%),PC1 (100.0%)_abs_dif
1,None,-0.400000,0.000000
2,None,-0.200000,0.200000
3,None,0.400000,0.800000
4,PC1,0.800000,1.200000


As a real-world example, you can load our γ-secretase substrate prediction dataset containing substrates (positive samples, 1) and a redundancy-reduced set of single-span type I transmembrane proteins with unknown substrates status (unlabeled samples, 2):

In [3]:
df_seq = aa.load_dataset(name="DOM_GSEC_PU")
labels = df_seq["label"].to_numpy()
n_pos = sum([x == 1 for x in labels])   # Get number of positive samples
aa.display_df(df=df_seq.tail(5), show_shape=True, n_cols=5)

DataFrame shape: (5, 8)


,entry,sequence,label,tmd_start,tmd_stop
690,P60852,MAGGSATTWGYPVAL...LSQTWAQKLWESNRQ,2,602,624
691,P20239,MARWQRKASVSSPCG...FICYLYKKRTIRFNH,2,684,703
692,P21754,MELSYRLFICLLLWG...TRRCRTASHPVSASE,2,387,409
693,Q12836,MWLLRCVLLCVSLSL...LAVKKQKSCPDQMCQ,2,506,528
694,Q8TCW7,MEQIWLLLLLTIRVL...PTSLVLNGIRNPVFD,2,374,396


Using the respective features, we can create a feature matrix and obtain 'reliable' non-substrates by dPULearn:

In [4]:
df_feat = aa.load_features(name="DOM_GSEC")
sf = aa.SequenceFeature()
df_parts = sf.get_df_parts(df_seq=df_seq)
X = sf.feature_matrix(features=df_feat["feature"], df_parts=df_parts)

# Number of positive (1) and unlabeled (2) samples
print(pd.Series(labels).value_counts())
dpul.fit(X=X, labels=labels, n_unl_to_neg=n_pos)
df_pu = dpul.df_pu_
new_labels = dpul.labels_ 

# Number of updated labels containing reliable negatives (0)
print(pd.Series(new_labels).value_counts())

# Show only selected entries
df = df_pu[df_pu["selection_via"].str.contains("PC", na=False)]
aa.display_df(df=df, show_shape=True, n_rows=20, n_cols=5)

2    631
1     63
Name: count, dtype: int64
2    568
1     63
0     63
Name: count, dtype: int64
DataFrame shape: (63, 15)


,selection_via,PC1 (56.2%),PC2 (7.4%),PC3 (2.9%),PC4 (2.8%)
81,PC3,0.033600,0.007300,0.098200,-0.007800
82,PC7,0.033400,-0.041100,0.033500,-0.005200
84,PC1,0.021000,-0.047800,0.075200,-0.005400
90,PC4,0.039000,-0.032000,-0.001300,0.110900
95,PC2,0.032000,-0.082100,0.025800,-0.037700
109,PC1,0.026100,-0.058500,0.075700,-0.020900
149,PC1,0.026500,-0.038000,0.019100,0.045500
158,PC1,0.023500,-0.060700,0.054000,0.000900
161,PC1,0.025900,0.031400,0.044900,0.055400
169,PC1,0.026500,-0.009900,0.012500,-0.016700


Since ``dPULearn().fit()`` returns the fitted model, list comprehension can be utilized to create results for various settings of a ``n_componentes``. If given as a float > 0 and < 1, this parameter represents the percentage of total variance to be retained by principal component analysis (PCA).

In [5]:
list_labels = [dpul.fit(X=X, labels=labels, n_unl_to_neg=n_pos, n_components=i).labels_ for i in [0.6, 0.7, 0.8, 0.9, 0.95]]

As alternative to ``PCA-based identification`` of negatives, ``distance-based identification`` can be performed using distance metrics including 'euclidean', 'manhattan', or 'cosine' distance. A DataFrame with the 

In [6]:
df_pu = dpul.fit(X=X, labels=labels, n_unl_to_neg=n_pos, metric="euclidean").df_pu_
aa.display_df(df_pu.sort_values(by="selection_via"), n_rows=10, show_shape=True)

DataFrame shape: (694, 3)


,selection_via,euclidean_dif,euclidean_abs_dif
84,euclidean,3.480700,3.480700
505,euclidean,3.232700,3.232700
509,euclidean,3.336300,3.336300
526,euclidean,3.389700,3.389700
533,euclidean,3.363900,3.363900
542,euclidean,3.075000,3.075000
546,euclidean,3.162500,3.162500
548,euclidean,3.111900,3.111900
552,euclidean,3.288600,3.288600
553,euclidean,3.620800,3.620800


Using ``PCA-based identification``, 'df_pu' provides the principal component (PC) values for all used PC and offers a label indicating based on which PC the respective negative samples was identified on:   

In [7]:
df_pu = dpul.fit(X=X, labels=labels, n_unl_to_neg=n_pos, n_components=0.8).df_pu_
aa.display_df(df_pu.sort_values(by="selection_via"), n_rows=n_pos+1, n_cols=4, show_shape=True)

DataFrame shape: (694, 15)


,selection_via,PC1 (56.2%),PC2 (7.4%),PC3 (2.9%)
497,PC1,0.022500,-0.051200,0.013400
615,PC1,0.026100,-0.053300,0.099300
406,PC1,0.025400,-0.030800,0.027200
446,PC1,0.026200,-0.013700,0.054500
455,PC1,0.026600,-0.052100,0.089500
468,PC1,0.025600,-0.068800,0.011800
471,PC1,0.025000,-0.005500,0.083500
668,PC1,0.023200,-0.016900,0.076500
605,PC1,0.025800,-0.054500,0.006700
505,PC1,0.023100,-0.048400,0.033900
